In [1]:
import pandas as pd
import sys
import pickle
import importlib
import os
import numpy as np
import scrapbook as sb
from sentence_transformers import SentenceTransformer

# Ensure project root is on sys.path (required for papermill fresh kernels)
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.insert(0, current_dir)

import free_entailments_algorithm_utils as fea

In [2]:
iteration_number = 1
input_csv_path = "labeled_pairs/Results_DS_BtoS_iteration_0.csv"
df_clause_path = None
embedding_cache_path = None
test = True
remaining_llm_calls_path = None
unlabeled_pairs_path = None
sent_frac = 0.5
budget = 0.00

In [3]:
# Parameters
iteration_number = 0
input_csv_path = "labeled_pairs/Results_DS_BtoS_iteration_0.csv"
df_clause_path = "fea_iterations\\loop_data/df_clause.pkl"
embedding_cache_path = "fea_iterations\\loop_data/embedding_cache.pkl"
test = False
remaining_llm_calls_path = None
unlabeled_pairs_path = "fea_iterations\\loop_data/unlabeled_pairs.pkl"
sent_frac = 0.5
budget = 0.0


In [4]:
pipeline_data = fea.load_pipeline_data(
    df_clause_path=df_clause_path,
    embedding_cache_path=embedding_cache_path,
    test=test,
    remaining_llm_calls_path=remaining_llm_calls_path,
    unlabeled_pairs_path=unlabeled_pairs_path,
    iteration_number=iteration_number,
)

df_clause = pipeline_data['df_clause']
embedding_cache_finetuned = pipeline_data['embedding_cache']
remaining_llm_calls = pipeline_data['remaining_llm_calls']
unlabeled_pairs = pipeline_data['unlabeled_pairs']


PARAMETER VALUES AFTER PAPERMILL INJECTION:
iteration_number = 0
test = False
remaining_llm_calls_path = None
df_clause_path = fea_iterations\loop_data/df_clause.pkl

✓ Loaded df_clause: 63909 rows


✓ Loaded embedding cache: 63909 embeddings
✓ Loaded unlabeled_pairs: 1000000 rows
✓ All data loaded from pickle files


# Task 1: Seting up dataframes and Running FEA

In [5]:
df_llm_original = pd.read_csv(input_csv_path)

# If 'verdict' column already exists (e.g. from process_llm_results_bidirectional),
# use it directly instead of recomputing via add_verdict (which only sees one-way
# conclusions and would overwrite correct bidirectional verdicts).
if 'verdict' in df_llm_original.columns and df_llm_original['verdict'].notna().any():
    df_llm = df_llm_original
    print(f"Using existing 'verdict' column ({(df_llm['verdict']=='YES').sum()} YES, {(df_llm['verdict']=='NO').sum()} NO)")
else:
    df_llm = fea.add_verdict(
        df_llm_original,
        id1_col='sentence_id_1',
        id2_col='sentence_id_2',
        conclusion_col='llm_conclusion_12',
        positive_label='YES'
    )

if test:
    df_llm_remaining = fea.add_verdict(
        remaining_llm_calls,
        id1_col='sentence_id_1',
        id2_col='sentence_id_2',
        conclusion_col='llm_conclusion_12',
        positive_label='YES'
    )

Using existing 'verdict' column (2 YES, 8 NO)


In [6]:
df_labeled = fea.merge_pairwise_texts(
    df1 = df_clause,
    df2 = df_llm,
    df1_cols = ['sentence_id', 'sentence'],
    df2_cols = ['sentence_id_1', 'sentence_id_2', 'verdict']
)
df_labeled.head()

,id1,id2,text1,text2,verdict
0,B0448006p,B1089003p,The emphasis on a singular sovereign power in ...,The accountability of the sovereign to God emp...,NO
1,B1089003p,B0448006p,The accountability of the sovereign to God emp...,The emphasis on a singular sovereign power in ...,NO
2,B0674004p,S0051696006p,Maintaining order and justice is essential for...,Maintaining democratic oversight is crucial to...,NO
3,S0051696006p,B0674004p,Maintaining democratic oversight is crucial to...,Maintaining order and justice is essential for...,NO
4,B0427001sc,B0596001sc,Popular Estates are essential for a just monar...,Lawful political authority is essential for so...,NO


In [7]:
if test:
    df_predict = fea.merge_pairwise_texts(
        df1 = df_clause,
        df2 = df_llm_remaining,
        df1_cols = ['sentence_id', 'sentence'],
        df2_cols = ['sentence_id_1', 'sentence_id_2']
    )
else:
    # At 20M+ pairs, we skip merge_pairwise_texts on the full pool
    # (pairs were already validated at generation, text not needed until
    #  the small df_final at the end). Just remove already-labeled pairs.
    df_predict = fea.setminus(
        df_big= unlabeled_pairs,
        df_small= df_labeled,
        id_cols = ['id1', 'id2']
    )
    df_predict['verdict'] = np.nan

df_predict.head()


Set difference: 1,000,000 - 10 = 1,000,000 rows


,id1,id2,verdict
0,B0001001p,B0005008p,NaN
1,B0001001p,B0020004p,NaN
2,B0001001p,B0083007p,NaN
3,B0001001p,B0115001p,NaN
4,B0001001p,B0119005p,NaN


## Embedding All Sentences

In [8]:
## Patches an error later on with kwargs
import transformers.utils.hub
import transformers.tokenization_utils_base

def _safe_list_templates(*args, **kwargs):
    return []

transformers.utils.hub.list_repo_templates = _safe_list_templates
print(" - Patched transformers.utils.hub")

# The library had already imported the broken function here, so we must update it.
transformers.tokenization_utils_base.list_repo_templates = _safe_list_templates
print(" - Patched transformers.tokenization_utils_base")

print("\nSUCCESS: The 404 error is now blocked.")

 - Patched transformers.utils.hub
 - Patched transformers.tokenization_utils_base

SUCCESS: The 404 error is now blocked.


## Test and Validation Subsamples

In [9]:
# Keep only entailed pairs from sent
df_obs_ent = df_labeled.loc[df_labeled['verdict'] == 'YES']
df_obs_ent.head()

,id1,id2,text1,text2,verdict
8,B0083004p,B0132002p,The legitimacy of governance is rooted in the ...,The essence of legitimate governance lies in t...,YES
9,B0132002p,B0083004p,The essence of legitimate governance lies in t...,The legitimacy of governance is rooted in the ...,YES


In [10]:
df_candidates = fea.add_equivalents_from_pairs(
    df3=df_obs_ent,
    df4=df_predict,
    df3_cols=["id1", "id2"],
    df4_cols=["id1", "id2"],
    new_cols=("equivalents1", "equivalents2"),
    include_self=False,
)

df_candidates = fea.add_alpha_weight_column(
    df = df_candidates,
    list_col1 = 'equivalents1',
    list_col2 = 'equivalents2',
    new_col = "alpha"
)

In [11]:
df_labeled = fea.add_equivalents_from_pairs(
    df3=df_obs_ent,
    df4=df_labeled,
    df3_cols=["id1", "id2"],
    df4_cols=["id1", "id2"],
    new_cols=("equivalents1", "equivalents2"),
    include_self=False,  # keep the ID itself in the list
)

df_labeled = fea.add_alpha_weight_column(
    df = df_labeled,
    list_col1 = 'equivalents1',
    list_col2 = 'equivalents2',
    new_col = "alpha"
)

## Equivalence Classes

In [12]:
# Produce set of all pairs of clauses i/j with k in the class of j/i
df_crossed = fea.build_equiv_pair_candidates(
    df = df_candidates,
    id1_col = "id1",
    id2_col = "id2",
    equiv1_col = "equivalents1",
    equiv2_col = "equivalents2",
)

# Retrieve clause sentences
df_crossed = fea.merge_pairwise_texts(
    df1 = df_clause,
    df2 = df_crossed,
    df1_cols = ['sentence_id', 'sentence'],
    df2_cols = ['id1', 'id2']
)

df_crossed.head()

,id1,id2,text1,text2,verdict
0,B0017003p,B0083004p,The establishment of a governing body or autho...,The legitimacy of governance is rooted in the ...,NaN
1,B0078002p,B0083004p,Filmer asserts that there is only one legitima...,The legitimacy of governance is rooted in the ...,NaN
2,B0085003p,B0132002p,Blind faith in a ruler is flawed because it di...,The essence of legitimate governance lies in t...,NaN
3,B0256001p,B0132002p,Nations can indeed meet and confer sovereignty...,The essence of legitimate governance lies in t...,NaN
4,B0293009p,B0132002p,The structure of Parliament minimizes the risk...,The essence of legitimate governance lies in t...,NaN


In [13]:
df_labeled_crossed = fea.build_equiv_pair_candidates(
    df = df_labeled,
    id1_col = "id1",
    id2_col = "id2",
    equiv1_col = "equivalents1",
    equiv2_col = "equivalents2",
)

# Retrieve clause sentences
df_labeled_crossed = fea.merge_pairwise_texts(
    df1 = df_clause,
    df2 = df_labeled_crossed,
    df1_cols = ['sentence_id', 'sentence'],
    df2_cols = ['id1', 'id2']
)

df_labeled_crossed.head()

,id1,id2,text1,text2,verdict
0,B0083004p,B0083004p,The legitimacy of governance is rooted in the ...,The legitimacy of governance is rooted in the ...,NaN
1,B0132002p,B0132002p,The essence of legitimate governance lies in t...,The essence of legitimate governance lies in t...,NaN
2,B0132002p,B0132002p,The essence of legitimate governance lies in t...,The essence of legitimate governance lies in t...,NaN
3,B0083004p,B0083004p,The legitimacy of governance is rooted in the ...,The legitimacy of governance is rooted in the ...,NaN


## Running FEA

In [14]:
df_final, fig_html = fea.run_fea_papermill(
    iteration_number=iteration_number,
    df_candidates=df_candidates,
    df_crossed=df_crossed,
    df_labeled=df_labeled,
    df_labeled_crossed=df_labeled_crossed,
    df_obs_ent=df_obs_ent,
    df_clause=df_clause,
    embedding_cache=embedding_cache_finetuned,
)

Executing FreeEntailmentAlgorithm.ipynb for iteration 0...


Executing:   0%|          | 0/33 [00:00<?, ?cell/s]

✓ Retrieved outputs:
  - df_final: 2663 rows
  - fig_html: HTML plot (13911 chars)


In [15]:
df_final.head()

,id1,id2,text1,text2,entailment_probability
444,B0004001p,B0399006p,Usurpation cannot be considered a legitimate m...,Just as God cannot create another equal to Him...,0.185158
541,B0004005p,B0882004p,The ability to govern with supreme power impli...,The power of kings is derived from a higher au...,0.413684
548,B0004006p,B0235005p,"The notion that any individual, regardless of ...",The king cannot possess power over the law,0.295776
555,B0004006p,B0564007p,"The notion that any individual, regardless of ...",A unilateral action by one part of the populac...,0.337070
566,B0004007p,B0167007p,The notion that any individual can be deemed a...,The idea that a king can grant privileges to t...,0.387469


# Task 2: Cleaning LLM Calls

In [16]:
# Cap at 100k pairs max — send ALL pairs above threshold (no random subsampling)
MAX_LLM_PAIRS = 50

df_final = df_final.reset_index(drop=True)

if len(df_final) > MAX_LLM_PAIRS:
    df_to_llm = df_final.sample(n=MAX_LLM_PAIRS, random_state=42)
    print(f"Capped df_to_llm at {MAX_LLM_PAIRS:,} (from {len(df_final):,} above threshold)")
else:
    df_to_llm = df_final.copy()
    print(f"Sending all {len(df_to_llm):,} pairs above threshold to LLM")

Capped df_to_llm at 50 (from 2,663 above threshold)


In [17]:
df_to_llm = fea.format_df_to_llm(df_to_llm, df_clause=df_clause, id_col='sentence_id', text_col='sentence')
df_to_llm.head()

,sentence_id_2,sentence_id_1,sentence_text_2,argument_id_2,sentence_text_1,argument_id_1,score
1474,B0564002sc,B0125001sc,It is not the case that a king can be legitima...,B0564,Ruler authority does not depend on the consent...,B0125,0.403788
124,B0143007p,B0123002p,The legitimacy of any government is contingent...,B0143,Legitimate power arises from the consent of th...,B0123,0.295776
1127,S0004096006p,B0591002p,The king must govern with the consent of the g...,S00040,"A king, like any other individual, should be h...",B0591,0.319019
1616,B0810001sc,B0349001sc,Kings can influence religious matters,B0810,The peers of the kingdom hold authority over t...,B0349,0.244741
1360,S0023811003p,B1079004p,The importance of upholding the rule of law em...,S00238,Punishment is crucial for maintaining order an...,B1079,0.273547


In [18]:
df_to_llm.shape

(50, 7)

# Next loop:

In [19]:
result = fea.finalize_pipeline_iteration(
    test=test,
    df_to_llm=df_to_llm,
    iteration_number=iteration_number,
    remaining_llm_calls=remaining_llm_calls,
    remaining_llm_calls_path=remaining_llm_calls_path,
    unlabeled_pairs=unlabeled_pairs,
    unlabeled_pairs_path=unlabeled_pairs_path,
)

remaining_llm_calls = result['remaining_llm_calls']
unlabeled_pairs = result['unlabeled_pairs']

✓ Saved 50 pairs to fea_iterations/llm_batch_iter_0.csv for LLM processing
✓ LLM labeled pairs updated: 60 total in labeled_pairs/llm_labeled_pairs.csv


✓ Removed 50 pairs from unlabeled_pairs
✓ Remaining pairs for future iterations: 999950
✓ Saved updated unlabeled_pairs to fea_iterations\loop_data/unlabeled_pairs.pkl


✓ Glued df_to_llm to scrapbook for FEA_Loop retrieval

Iteration 0 complete
